In [1]:
import redis

# Подключаемся к Redis (который запущен через Docker на localhost)
conn = redis.Redis(
    host='localhost',  # Локальный хост, так как Redis работает в Docker
    port=6379,         # Стандартный порт Redis
    decode_responses=True
)

# Проверка соединения с Redis
def check_redis_connection():
    try:
        conn.ping()  # Пингируем Redis, чтобы проверить соединение
        print("Connected to Redis successfully!")
    except redis.ConnectionError as e:
        print(f"Failed to connect to Redis: {e}")

check_redis_connection()


Connected to Redis successfully!


In [2]:
from time import sleep
import time

# Обработка задач без RQ (последовательно)
def process_task_sequential(task_id):
    sleep(1)  # Имитируем выполнение задачи
    return f"Task {task_id} completed."

# Замер времени без RQ
start_time = time.time()
results_sequential = []
for i in range(5):  # Уменьшаем количество задач для быстрого теста
    results_sequential.append(process_task_sequential(i))
end_time = time.time()
elapsed_time_sequential = end_time - start_time
print(f"Time taken sequentially: {elapsed_time_sequential} seconds.")


Time taken sequentially: 5.00351881980896 seconds.


In [3]:
!pip install dill


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install celery redis


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import time
from celery import Celery

# Создаем приложение Celery с настройками для Redis
app = Celery('tasks', broker='redis://localhost:6379/0', backend='redis://localhost:6379/0')

# Функция, которую будем использовать как задачу
@app.task
def process_task(task_id):
    time.sleep(1)
    return f"Task {task_id} completed."

# Запуск задачи через Celery (последовательно)
start_time = time.time()
results_seq = [process_task(i) for i in range(5)]
end_time = time.time()
elapsed_time_seq = end_time - start_time
print(f"Time taken without Celery (sequential): {elapsed_time_seq} seconds.")

# Запуск задач через Celery (асинхронно)
start_time = time.time()
results_celery = []

for i in range(5):
    result = process_task.apply_async(args=[i])
    results_celery.append(result)

# Получаем результаты выполнения задач
for result in results_celery:
    print(f"Result: {result.result}")  # Теперь Celery должен вернуть результат

end_time = time.time()
elapsed_time_celery = end_time - start_time
print(f"Time taken with Celery (queue): {elapsed_time_celery} seconds.")


Time taken without Celery (sequential): 5.012699842453003 seconds.
Result: None
Result: None
Result: None
Result: None
Result: None
Time taken with Celery (queue): 0.11936664581298828 seconds.


In [8]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/9f/6e/264673e64001b99d747aff5a288eca82826c024437a3694e19aed1decf46/matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/a1/35/c2de8823211d07e8a79ab018ef03960716c5dff6f4d5bff5af87fd682992/contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/46/45/a498b5291f6c0d91b2394b1ed7447442a57d1c9b9cf8f439aee3c316a56e/fonttools-4.55.3-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/168.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/168.5 kB ? eta -:-


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import time
from celery import Celery
import matplotlib.pyplot as plt

# Создаем приложение Celery с настройками для Redis
app = Celery('tasks', broker='redis://localhost:6379/0', backend='redis://localhost:6379/0')

# Функция, которую будем использовать как задачу
@app.task
def process_task(task_id):
    time.sleep(0.1)  # Уменьшаем задержку для ускорения эксперимента
    return f"Task {task_id} completed."

def run_experiment(data_sizes):
    results = {"sequential": [], "celery": []}

    for size in data_sizes:
        # Эксперимент без Celery (последовательно)
        start_time = time.time()
        results_seq = [process_task(i) for i in range(size)]
        end_time = time.time()
        elapsed_time_seq = end_time - start_time
        results["sequential"].append(elapsed_time_seq)

        # Эксперимент с Celery (асинхронно)
        start_time = time.time()
        results_celery = []
        for i in range(size):
            result = process_task.apply_async(args=[i])
            results_celery.append(result)

        # Получаем результаты выполнения задач
        for result in results_celery:
            result.get()  # Получаем результат, что тоже замедляет эксперимент

        end_time = time.time()
        elapsed_time_celery = end_time - start_time
        results["celery"].append(elapsed_time_celery)

    return results

# Задаем различные размеры данных для эксперимента
data_sizes = [5, 10, 20, 50, 100]

# Запуск эксперимента
experiment_results = run_experiment(data_sizes)

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(data_sizes, experiment_results["sequential"], label='Sequential (No Celery)', marker='o')
plt.plot(data_sizes, experiment_results["celery"], label='Celery (Async)', marker='o')
plt.xlabel('Number of Tasks')
plt.ylabel('Execution Time (seconds)')
plt.title('Execution Time Comparison: Sequential vs Celery')
plt.legend()
plt.grid(True)
plt.show()


KeyboardInterrupt: 